In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

# 1. Data pre-processing in BigQuery
1. Get one week of data from this public available BiqQuery table: gdelt-bq.gdeltv2.geg_gcnlapi
2. Create hosts table, articles table and entities table, with node ids assigned
3. Create 2 edge index tables: host to article and article to entity


# 2. Read pre-processed data from BigQuery into Dataframes
Hosts table, article table, entities table, and edge indices between them.

In [2]:
# BigQuery api client
bq = bigquery.Client(location="US")


In [3]:
def nodes_tables_from_bq(bq):
    host_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.hosts_table` 
        """
    article_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.articles_table` 
        """
    entity_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.entities_table` 
        """
    query_job = bq.query(host_query, location="US") 
    hosts = query_job.to_dataframe() 
    query_job = bq.query(article_query, location="US") 
    articles = query_job.to_dataframe() 
    query_job = bq.query(entity_query, location="US") 
    entities = query_job.to_dataframe() 
    return hosts, articles, entities

In [4]:
# use local csv files when possible 
def load_nodes_tables(bq):
    exists_hosts = os.path.isfile('data/hosts_table.csv')
    exists_articles = os.path.isfile('data/articles_table.csv')
    exists_entities = os.path.isfile('data/entities_table.csv')
    if exists_hosts and exists_articles and exists_entities:
        print("Read from local csv files.")
        hosts = pd.read_csv('data/hosts_table.csv')
        articles = pd.read_csv('data/articles_table.csv')
        entities = pd.read_csv('data/entities_table.csv')
    else:
        print("Read from BigQuery.")
        hosts, articles, entities = nodes_tables_from_bq(bq)
        hosts.to_csv('data/hosts_table.csv')
        articles.to_csv('data/articles_table.csv')
        entities.to_csv('data/entities_table.csv')
    
    return hosts, articles, entities

In [5]:
# load nodes tables into dataframes
hosts_table, articles_table, entities_table = load_nodes_tables(bq)
print(hosts_table.shape, articles_table.shape, entities_table.shape)


Read from local csv files.
(20033, 3) (1048852, 7) (1289112, 4)


In [6]:
print(hosts_table.shape, articles_table.shape, entities_table.shape)

(20033, 3) (1048852, 7) (1289112, 4)


In [7]:
# sort the dataframes by node ids
hosts_table = hosts_table.sort_values(by=['host_node_id'], ascending=True).reset_index(drop=True)
articles_table = articles_table.sort_values(by=['article_node_id'], ascending=True).reset_index(drop=True)
entities_table = entities_table.sort_values(by=['entity_node_id'], ascending=True).reset_index(drop=True)


In [8]:
display(hosts_table.head(3))
display(articles_table.head(3))
display(entities_table.head(3))


,Unnamed: 0,host_node_id,host
0,3605,0,www.berchtesgadener-anzeiger.de
1,18421,1,www.toponline.ch
2,14060,2,news.yahoo.com


,Unnamed: 0,article_node_id,host,url,lang,magnitude,score
0,63332,0,www.workersliberty.org,https://www.workersliberty.org/index.php/audio,en,15.9,0.0
1,695255,1,www.wkyc.com,https://www.wkyc.com/article/news/nation-world...,en,24.6,-0.4
2,678766,2,townhall.com,https://townhall.com/tipsheet/mattvespa/2023/1...,en,13.2,-0.4


,Unnamed: 0,entity_node_id,mid,type
0,710406,0,/m/0138vk,LOCATION
1,239295,1,/m/02phmc9,LOCATION
2,1224603,2,/m/04gsnk1,ORGANIZATION


## 2.1 Nodes
Nodes Features:
1. Hosts (News Outlets) - News Outlet (identified by host) - use random node feature for host, because hosts do not have rich features
2. Articles (identified by url) - lang, magnitude, score
3. Entity (identified by mid) - type


In [9]:
# use random node feature for host, because hosts do not have rich features
random_feature_dim = 32
host_features = torch.randn(hosts_table.shape[0], random_feature_dim)  # Random features for each node
host_features.shape

torch.Size([20033, 32])

In [10]:
# features for article: lang, magnitude, score
unique_langs, article_langs = np.unique(articles_table['lang'].values, return_inverse=True)
article_features = pd.get_dummies(article_langs)
article_features['magniture'] = articles_table['magnitude']
article_features['score'] = articles_table['score']
article_features = torch.from_numpy(article_features.values).to(torch.float)
article_features.shape

torch.Size([1048852, 12])

In [11]:
# features for entities: type
unique_types, entity_types = np.unique(entities_table['type'].values, return_inverse=True)
entity_features = pd.get_dummies(entity_types)
entity_features = torch.from_numpy(entity_features.values).to(torch.float)
entity_features.shape

torch.Size([1289112, 7])

## 2.2 Edge Indices
Two types of edges: 
1. host to article
2. article to entity


In [12]:
def edge_indices_from_bq(bq):
    host_article_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.edge_index_host_to_article` 
        """
    article_entity_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.edge_index_article_to_entity` 
        """
    query_job = bq.query(host_article_query, location="US") 
    host_article = query_job.to_dataframe() 
    query_job = bq.query(article_entity_query, location="US") 
    article_entity = query_job.to_dataframe() 

    return host_article, article_entity
    

In [13]:
# use local csv files when possible 
def load_edge_indices(bq):
    exists_host_article = os.path.isfile('data/edge_index_host_to_article.csv')
    exists_article_entity = os.path.isfile('data/edge_index_article_to_entity.csv')

    if exists_host_article and exists_article_entity:
        print("Read from local csv files.")
        host_article = pd.read_csv('data/edge_index_host_to_article.csv')
        article_entity = pd.read_csv('data/edge_index_article_to_entity.csv')
    else:
        print("Read from BigQuery.")
        host_article, article_entity = edge_indices_from_bq(bq)
        host_article.to_csv('data/edge_index_host_to_article.csv')
        article_entity.to_csv('data/edge_index_article_to_entity.csv')
    
    return host_article, article_entity

In [15]:
# load edge indices into dataframes
host_article, article_entity = load_edge_indices(bq)
print(host_article.shape, article_entity.shape)

Read from local csv files.
(1048852, 2) (39171153, 2)


In [22]:
# construct `edge_index` in COO format
# host - article
host_index = torch.from_numpy(host_article['host_node_id'].values)
article_index = torch.from_numpy(host_article['article_node_id'].values)
edge_index_host_to_article = torch.stack([host_index, article_index], dim=0)
assert edge_index_host_to_article.size() == (2, host_article.shape[0])

# article - entity
article_index = torch.from_numpy(article_entity['article_node_id'].values)
entity_index = torch.from_numpy(article_entity['entity_node_id'].values)
edge_index_article_to_entity = torch.stack([article_index, entity_index], dim=0)
assert edge_index_article_to_entity.size() == (2, article_entity.shape[0])

print(edge_index_host_to_article.shape, edge_index_article_to_entity.shape)

torch.Size([2, 1048852]) torch.Size([2, 39171153])


# 3. Create HeteroData object

In [23]:
hetero_data = HeteroData()
# Save node indices:
hetero_data['host'].node_id = torch.arange(hosts_table.shape[0])
hetero_data['article'].node_id = torch.arange(articles_table.shape[0])
hetero_data['entity'].node_id = torch.arange(entities_table.shape[0])
# Add the node features and edge indices:
hetero_data['host'].x = host_features
hetero_data['article'].x = article_features
hetero_data['entity'].x = entity_features
# edges
hetero_data['host', 'publishes', 'article'].edge_index = edge_index_host_to_article
hetero_data['article', 'mentions', 'entity'].edge_index = edge_index_article_to_entity
hetero_data

HeteroData(
  host={
    node_id=[20033],
    x=[20033, 32],
  },
  article={
    node_id=[1048852],
    x=[1048852, 12],
  },
  entity={
    node_id=[1289112],
    x=[1289112, 7],
  },
  (host, publishes, article)={ edge_index=[2, 1048852] },
  (article, mentions, entity)={ edge_index=[2, 39171153] }
)

In [24]:
T.ToUndirected()(hetero_data)

HeteroData(
  host={
    node_id=[20033],
    x=[20033, 32],
  },
  article={
    node_id=[1048852],
    x=[1048852, 12],
  },
  entity={
    node_id=[1289112],
    x=[1289112, 7],
  },
  (host, publishes, article)={ edge_index=[2, 1048852] },
  (article, mentions, entity)={ edge_index=[2, 39171153] },
  (article, rev_publishes, host)={ edge_index=[2, 1048852] },
  (entity, rev_mentions, article)={ edge_index=[2, 39171153] }
)

In [ ]:
def read_geg_from_bq(dataset='week'):
    bq = bigquery.Client(location="US")
    dataset_1_month = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.geg_with_mid` 
        WHERE date between TIMESTAMP("2023-11-01") and TIMESTAMP("2023-11-30")
        """
    dataset_1_week = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.geg_with_mid` 
        WHERE date between TIMESTAMP("2023-11-19") and TIMESTAMP("2023-11-25")
        """
    
    query = dataset_1_month if dataset == 'month' else dataset_1_week

    query_job = bq.query(query, location="US") 
    df = query_job.to_dataframe() 
    return df
    

In [ ]:
def load_one_week_data():
    df = None
    exists = os.path.isfile('data/geg_1_week.csv')
    if exists:
        print("read from local file data/geg_1_week.csv")
        df = pd.read_csv('data/geg_1_week.csv')
    else:
        print("read from BigQuery")
        df = read_geg_from_bq('week')
        df.to_csv('data/geg_1_week.csv')
    
    return df
    

In [ ]:
geg = load_one_week_data()

print(geg.shape)

In [ ]:
geg.sample(5)

In [ ]:
# Create a mapping from unique news outlet (host) to range (0, num_news_outlets):
unique_news_outlets = geg['host'].unique()
unique_news_outlets = pd.DataFrame(data={
    'host': unique_news_outlets,
    'node_id': pd.RangeIndex(len(unique_news_outlets)),
})

print("Mapping of hosts to node ids:")
print("==========================================")
print(unique_news_outlets.head())
print()

In [ ]:
# Assign unique node ids to hosts
unique_hosts, host_node_ids = np.unique(geg['host'].values, return_inverse=True)
geg['host_node_id'] = host_node_ids


In [ ]:
# Assign unique node ids to articles, which are identified by urls
unique_articles, article_node_ids = np.unique(geg['url'].values, return_inverse=True)
geg['article_node_id'] = article_node_ids

In [ ]:
# Assign unique node ids to entities, which are identified by mids
unique_entities, entity_node_ids = np.unique(geg['mid'].values, return_inverse=True)
geg['entity_node_id'] = entity_node_ids

In [ ]:
print(len(unique_hosts), len(unique_articles), len(unique_entities))

unique_hosts

In [ ]:
geg.head()

In [ ]:
host_features.shape

In [ ]:
geg.shape

In [ ]:
usa_today = geg[geg['host'].str.contains('usatoday')]


In [ ]:
usa_today.shape

In [ ]:
nytimes = geg[geg['host'].str.contains('nytimes')]

In [ ]:
nytimes.shape